In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


from shutil import copyfile
import torch
from torch.utils.data import Dataset
from transformers import (InputFeatures)
from transformers import AutoTokenizer
from typing import Dict
from transformers import InputFeatures, T5ForConditionalGeneration, T5TokenizerFast

import pickle
import gc

In [2]:
# copyfile(src = "C:/Users/Maciek/Documents/Studia/Magisterka/GitHub/BalancingMethodsNLP/Proof of Concept/utils.py", dst = "/working/utils.py")
# copyfile(src = "/kaggle/input/balancing-script/Class Balancing.py", dst = "../working/Class_Balancing.py")
from utils import load_documents, load_datasets, annotations_from_jsonl, Annotation
from Class_Balancing import *

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# data_root = r'C:\Users\Maciek\Documents\Studia\Magisterka\eraser'
# documents = load_documents(data_root)

# train, val, test = load_datasets(data_root)

In [4]:
# def get_input(ann):
#     if len(ann.all_evidences())==0:
#         docid  = ann.annotation_id
#     else:
#         (docid,) = set(ev.docid for ev in ann.all_evidences())
#     doc = documents[docid]
#     input = ''
#     for sent in doc:
#         input += ' '.join(sent)
#     return input

In [5]:
# def print_color(text, evidences):
#     for ev in evidences:
#         t = "\033[95m" + ev+ '\x1b[0m'
#         if ev!='':
#             text = t.join(text.split(ev))
#     print(text)

In [6]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, evidences=None):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.evidences = evidences

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
#             token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])  

In [7]:
# def create_dataset(dataset, tokenizer):
#     targets = [1  if ann.classification != 'NEG' else 0 for ann in dataset]
#     evidences = [[ev.text for ev in ann.all_evidences()] for ann in dataset]
#     inputs = [get_input(ann) for ann in dataset]
    
#     if len(targets)==len(evidences)==len(inputs):
#         return TrainerDataset(inputs, targets, tokenizer, evidences)
#     else:
#         print("Something went wrong !!!!!")

In [8]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2)

In [9]:
# train_dataset = create_dataset(train, tokenizer)
# eval_dataset = create_dataset(val, tokenizer)
# test_dataset = create_dataset(test, tokenizer)

In [10]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
paraphraser_t5small = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")
tokenizer_t5small = T5TokenizerFast.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# import pickle
# dataset = "eraser_movie"

# filehandler = open(f'{dataset}_train.obj',"wb")
# pickle.dump(train_dataset,filehandler)
# filehandler.close()

# filehandler = open(f'{dataset}_eval.obj',"wb")
# pickle.dump(eval_dataset,filehandler)
# filehandler.close()

# filehandler = open(f'{dataset}_test.obj',"wb")
# pickle.dump(test_dataset,filehandler)
# filehandler.close()

In [13]:
# import nltk
# nltk.download('omw-1.4')

In [14]:
# imbalance = 0.1
# random_seed = 123
# np.random.seed(123)
# for syn_prob in [0.01, 0.02, 0.05, 0.1, 0.2]:
#     for i in range(20):
#         train_dataset_imbalanced = add_imbalance(train_dataset, imbalance, random_seed = np.random.randint(0,123))
#         train_dataset_synonym = balance_minority(train_dataset_imbalanced, replace_synonym, **{"prob":syn_prob}, random_seed = np.random.randint(0,123))

#         filehandler = open(fr'C:\Users\Maciek\Documents\Studia\Magisterka\Synonym_probability\{dataset}_{int(imbalance*100)}_synonym_{int(syn_prob*100)}%_{i}.obj',"wb")
#         pickle.dump(train_dataset_synonym,filehandler)
#         filehandler.close()

In [15]:
imbalance = 0.5
par_perc = 0.1
task = "eraser_movie"
np.random.seed(123)
for imbalance in [10, 20, 50]:
    for i in range(5):
        print(i+1)
        with open(fr"C:\Users\Maciek\Documents\Studia\Magisterka\GitHub\BalancingMethodsNLP\nlpaug\{task}_{imbalance}_imbalanced_{i}.obj", 'rb') as pickle_file:
            train_dataset_imbalanced = pickle.load(pickle_file)
        train_dataset_paraphrase = balance_minority(train_dataset_imbalanced, paraphrase_perc_sentences, False, random_seed = np.random.randint(0,123), **{"p":par_perc, "paraphraser":paraphraser_t5small, "tokenizer":tokenizer_t5small, "count":True})
        
        
        filehandler = open(fr'C:\Users\Maciek\Documents\Studia\Magisterka\GitHub\BalancingMethodsNLP\paraphrased\{task}_{imbalance}_paraphrase_{int(par_perc*100)}%_{i}.obj',"wb")
        pickle.dump(train_dataset_paraphrase,filehandler)
        filehandler.close()



1
218
3430
4170
5886
1617
2280
3923
4559
5213
1866
2302
3683
4083
5490
